In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

df=pd.read_csv('/Users/anaghaathavale/Downloads/compas-scores-raw.csv')

In [2]:
df=df.drop(['IsCompleted'],axis=1)
df=df.drop(['IsDeleted'],axis=1)
df=df.drop(['AssessmentType'],axis=1)
df=df.drop(['MiddleName'],axis=1)
df=df.drop(['Person_ID'],axis=1)
df=df.drop(['AssessmentID'],axis=1)
df=df.drop(['Case_ID'],axis=1)
df=df.drop(['Agency_Text'],axis=1)
df=df.drop(['LastName'],axis=1)
df=df.drop(['FirstName'],axis=1)
df=df.drop(['DateOfBirth'],axis=1)
df=df.drop(['Language'],axis=1)
df=df.drop(['RecSupervisionLevel'],axis=1)
df=df.drop(['RecSupervisionLevelText'],axis=1)
df=df.drop(['DisplayText'],axis=1)
df=df.drop(['RawScore'],axis=1)
df=df.drop(['DecileScore'],axis=1)
df=df.drop(['Screening_Date'],axis=1)
df=df.drop(['ScaleSet'],axis=1)

In [3]:
df = df.dropna()

In [4]:
indexAge = df[ (df['Ethnic_Code_Text'] == 'Native American')|(df['Ethnic_Code_Text'] == 'Oriental')|
              (df['Ethnic_Code_Text'] == 'Arabic')|(df['Ethnic_Code_Text'] == 'Other')|(df['Ethnic_Code_Text'] == 'Hispanic')|(df['Ethnic_Code_Text'] == 'Asian') ].index
df.drop(indexAge , inplace=True)

In [5]:
Sex_Code_Text={"Male":1, "Female":2}
df["Sex_Code_Text"]=df["Sex_Code_Text"].map(Sex_Code_Text)

In [6]:
Ethnic_Code_Text={"Caucasian":1, "African-American":2, "African-Am":2 }
df["Ethnic_Code_Text"]=df["Ethnic_Code_Text"].map(Ethnic_Code_Text)

#ScaleSet={"Risk and Prescreen":1, "All Scales":2}
#df["ScaleSet"]=df["ScaleSet"].map(ScaleSet)

LegalStatus={'Pretrial':1, 'Post Sentence':2 ,'Conditional Release':3,
 'Probation Violator':4, 'Parole Violator':5, 'Deferred Sentencing':6, 'Other':7}
df["LegalStatus"]=df["LegalStatus"].map(LegalStatus)

AssessmentReason={"Intake":1}
df["AssessmentReason"]=df["AssessmentReason"].map(AssessmentReason)

CustodyStatus={'Jail Inmate':1, 'Probation':2, 'Pretrial Defendant':3, 'Residential Program':4,
 'Prison Inmate':5, 'Parole':6}
df["CustodyStatus"]=df["CustodyStatus"].map(CustodyStatus)

MaritalStatus={'Single':1, 'Married':2, 'Significant Other':3, 'Divorced':4, 'Separated':5, 'Widowed':6,
 'Unknown':7}
df["MaritalStatus"]=df["MaritalStatus"].map(MaritalStatus)

ScoreText={"Low":1, "Medium":2, "High":3 }
df["ScoreText"]=df["ScoreText"].map(ScoreText)

In [7]:
ScaleSet_ID={22:1, 17:2}
df["ScaleSet_ID"]=df["ScaleSet_ID"].map(ScaleSet_ID)
Scale_ID={7:1, 8:2, 18:3}
df["Scale_ID"]=df["Scale_ID"].map(Scale_ID)

In [8]:
X = df.iloc[:,0:8].values
y = df.iloc[:,8].values
print(X[0:5])
print(y[0:5])


[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 3]
 [1 1 1 1 1 1 2 1]
 [1 1 1 1 1 1 2 2]]
[1 1 1 1 1]


In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

### One hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
Y = onehot_encoded
print(Y)

[0 0 0 ... 0 1 0]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(3, activation='relu')
  ])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=10)

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

model.save("compasModelFairness")

Metal device set to: Apple M1
Epoch 1/10


2024-02-09 12:27:02.497640: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-09 12:27:02.497812: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-02-09 12:27:02.613065: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-09 12:27:02.834337: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 4s 5ms/step - loss: 0.1954 - accuracy: 0.6511
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1876 - accuracy: 0.6511
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1862 - accuracy: 0.6511
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1853 - accuracy: 0.6511
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1846 - accuracy: 0.6511
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1842 - accuracy: 0.6511
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1840 - accuracy: 0.6511
Epoch 8/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1839 - accuracy: 0.6511
Epoch 9/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1837 - accuracy: 0.6511
Epoch 10/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1835 - accuracy: 0.6511


2024-02-09 12:27:41.030878: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Test loss: 0.1808125227689743
Test accuracy: 0.6573125720024109


2024-02-09 12:27:41.983192: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: compasModelFairness/assets


In [12]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

model.save("compasModel_Distillation_11HiddenNeurons")

Test loss: 0.1808125227689743
Test accuracy: 0.6573125720024109
INFO:tensorflow:Assets written to: compasModel_Distillation_11HiddenNeurons/assets
